In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# use your CRUD module/class
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# credentials for your Mongo user & DB
username = "joseph"
password = "password"
host = "127.0.0.1"
port = 27017
db_name = "aac"
collection_name = "animals"

# Connect to database via CRUD Module (pass all args positionally to match your constructor)
db = AnimalShelter(username, password, host, port, db_name, collection_name)

# initial unfiltered read
df = pd.DataFrame.from_records(db.read({}))

# drop _id (ObjectId) for DataTable; fill NaNs
if "_id" in df.columns:
    df.drop(columns=['_id'], inplace=True)
df = df.fillna("")

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add Grazioso Salvare’s logo and unique identifier
# (search a few common locations/case variants)
candidates = [
    "code_files/Grazioso Salvare Logo.png",
    "code_files/Grazioso Salvare Logo.PNG",
    "code_files/Grazioso_Salvare_Logo.png",
    "Grazioso Salvare Logo.png",
]
logo_path = None
for p in candidates:
    if os.path.exists(p):
        logo_path = p
        break
if not logo_path:
    raise FileNotFoundError("Place the Grazioso Salvare Logo PNG in code_files/ and update the path below.")

encoded_image = base64.b64encode(open(logo_path, "rb").read()).decode()

app.layout = html.Div([
    html.Div([
        html.A(
            html.Img(src=f"data:image/png;base64,{encoded_image}", style={"height": "48px"}),
            href="https://www.snhu.edu", target="_blank"
        ),
        html.Center(html.B(html.H1('CS-340 Dashboard — Joseph Toj')))
    ]),
    html.Hr(),

    # Interactive filtering options (radio buttons per spec)
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Reset (All)', 'value': 'reset'},
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster/Individual Tracking', 'value': 'disaster'},
            ],
            value='reset',
            labelStyle={'display': 'inline-block', 'marginRight': '16px'}
        )
    ),

    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),

        # DataTable usability (pagination/sort/filter) and single-row selection per spec
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[0]
    ),

    html.Br(),
    html.Hr(),

    # charts side-by-side: your pie (graph-id) and the geolocation map (map-id)
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id',   className='col s12 m6'),
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Mongo queries for each rescue filter (per dashboard spec)
def build_query(filter_type: str):
    if filter_type == "water":
        return {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
        }
    if filter_type == "mountain":
        return {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
        }
    if filter_type == "disaster":
        return {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300},
        }
    # reset (all)
    return {}

# filter -> update table data from Mongo
@app.callback(
    Output('datatable-id','data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    query = build_query(filter_type if filter_type else "reset")
    dff = pd.DataFrame.from_records(db.read(query))
    if "_id" in dff.columns:
        dff.drop(columns=["_id"], inplace=True)
    dff = dff.fillna("")
    return dff.to_dict('records')

# Display a pie chart of breeds based on the current table view
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData) if viewData is not None else pd.DataFrame()
    if dff.empty or "breed" not in dff.columns:
        fig = px.pie(values=[1], names=["No data"], title="Top Breeds")
    else:
        top = dff["breed"].value_counts().nlargest(10).reset_index()
        top.columns = ["breed", "count"]
        fig = px.pie(top, names="breed", values="count", title="Top Breeds (current filter)")
    return [dcc.Graph(figure=fig)]

#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in (selected_columns or [])]

# Geo-location chart (Leaflet) based on selected row in the table
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None:
        return
    dff = pd.DataFrame.from_dict(viewData)

    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13], dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

# Run app (JupyterLab inline or default port in Codio)
app.run_server()


Dash app running on https://saltpegasus-beachrussian-3000.codio.io/proxy/8050/
